In [57]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import TfidfTransformer
from keras.regularizers import l2, activity_l2
from sklearn.neighbors import KNeighborsClassifier
from keras.callbacks import Callback

class NBatchLogger(Callback):
    def __init__(self):
        self.seen = 0

    def on_epoch_end(self, batch, logs={}):
        self.seen += 1
        if self.seen % 50 == 0:
            print('epoch {}: loss = {}, val = {}'.format(self.seen, logs['loss'], logs['val_loss'])) 

In [43]:
train_X = pd.read_json('data/train_with_leaky_cv.json')
train_y = pd.read_json('data/train_interest.json')[0]
columns = train_X.columns
train_X_scaled = pd.DataFrame(StandardScaler().fit_transform(train_X), columns=columns)

In [44]:
column_usefulness = [('bc_price', 16597), ('manager_worths', 14684), ('price_by_NTACode', 13343), ('latitude', 12299), ('longitude', 12051), ('desc_words_length', 11477), ('manager_id', 11136), ('price_by_address1', 11125), ('Yday', 10884), ('address1', 10458), ('building_worths', 10298), ('building_id', 10142), ('desc_letters_count', 9832), ('real_minus_expected_price', 9815), ('price_by_area', 9742), ('real_over_expected_price', 9374), ('Day', 9066), ('worth_by_address1', 8782), ('hour', 7722), ('price_per_bathroom', 7590), ('desc_words_count', 7481), ('price_per_bed_and_bath', 7387), ('worth_by_NTACode', 6993), ('photos_count', 6730), ('NTACode', 6324), ('price_by_BoroCode', 6098), ('features_count', 5668), ('Wday', 4445), ('price_per_bedroom', 3903), ('sparse_nofee', 3056), ('bedrooms', 2833), ('sparse_furnished', 1865), ('bathrooms', 1702), ('sparse_laundryinbuilding', 1425), ('sparse_hardwoodfloors', 1132), ('sparse_laundryinunit', 1088), ('Month', 1006), ('sparse_exclusive', 771), ('sparse_dogsallowed', 770), ('BoroCode', 749), ('sparse_catsallowed', 718), ('sparse_prewar', 663), ('sparse_privateoutdoorspace', 634), ('sparse_reducedfee', 631), ('avenue', 595), ('street', 569), ('other_address', 553), ('sparse_commonoutdoorspace', 545), ('Zero_building_id', 534), ('sparse_elevator', 533), ('east', 524), ('sparse_doorman', 510), ('sparse_parkingspace', 506), ('sparse_dishwasher', 500), ('sparse_highspeedinternet', 476), ('sparse_terrace', 424), ('sparse_fitnesscenter', 423), ('sparse_loft', 413), ('sparse_outdoorspace', 406), ('sparse_actualapt', 399), ('sparse_roofdeck', 359), ('sparse_balcony', 346), ('sparse_diningroom', 318), ('sparse_simplex', 292), ('sparse_swimmingpool', 283), ('sparse_shorttermallowed', 263), ('sparse_wheelchairaccess', 248), ('west', 245), ('sparse_newconstruction', 238), ('sparse_garden', 231), ('sparse_multilevel', 201), ('sparse_patio', 199), ('sparse_hardwood', 198), ('worth_by_BoroCode', 179), ('sparse_fireplace', 168), ('sparse_highceiling', 159), ('sparse_petsok', 154), ('sparse_dryerinunit', 153), ('sparse_stainlesssteelappliances', 148), ('sparse_garage', 119), ('south', 118), ('sparse_storage', 118), ('sparse_outdoorareas', 117), ('sparse_lndrybldg', 113), ('sparse_liveinsuper', 110), ('sparse_new', 103), ('sparse_renovated', 99), ('sparse_photos', 88), ('sparse_highceilings', 79), ('sparse_onsitelaundry', 79), ('sparse_view', 78), ('sparse_flex3', 67), ('sparse_centrala', 66), ('sparse_concierge', 64), ('sparse_residentslounge', 63), ('sparse_assignedparkingspace', 62), ('sparse_allutilitiesincluded', 58), ('sparse_onsitegarage', 53), ('north', 46), ('sparse_lowrise', 44), ('sparse_washerinunit', 44)]
leaky = ['manager_worths', 'building_worths', 'price_by_area', 'real_minus_expected_price', 'real_over_expected_price', 'price_by_address1', 'worth_by_address1', 'price_by_BoroCode', 'worth_by_BoroCode', 'price_by_NTACode', 'worth_by_NTACode']
all_columns = ['BoroCode', 'Day', 'Month', 'NTACode', 'Wday', 'Yday', 'Zero_building_id', 'Zero_manager_id', 'address1', 'avenue', 'bathrooms', 'bc_price', 'bedrooms', 'building_id', 'building_worths', 'desc_letters_count', 'desc_words_count', 'desc_words_length', 'east', 'features_count', 'hour', 'latitude', 'longitude', 'manager_id', 'manager_worths', 'north', 'other_address', 'photos_count', 'price_by_BoroCode', 'price_by_NTACode', 'price_by_address1', 'price_by_area', 'price_per_bathroom', 'price_per_bed_and_bath', 'price_per_bedroom', 'real_minus_expected_price', 'real_over_expected_price', 'south', 'sparse_24', 'sparse_2fullbaths', 'sparse_actualapt', 'sparse_airconditioning', 'sparse_allutilitiesincluded', 'sparse_assignedparkingspace', 'sparse_attendedlobby', 'sparse_backyard', 'sparse_balconies', 'sparse_balcony', 'sparse_basementstorage', 'sparse_bikeroom', 'sparse_bikestorage', 'sparse_billiardsroom', 'sparse_brownstone', 'sparse_buildingcommonoutdoorspace', 'sparse_businesscenter', 'sparse_cable', 'sparse_catsallowed', 'sparse_centrala', 'sparse_centralac', 'sparse_centralair', 'sparse_chef', 'sparse_chefskitchen', 'sparse_children', 'sparse_childrensplayroom', 'sparse_cityview', 'sparse_commonbackyard', 'sparse_commongarden', 'sparse_commonoutdoorspace', 'sparse_commonparking', 'sparse_commonroofdeck', 'sparse_commonterrace', 'sparse_communityrecreationfacilities', 'sparse_concierge', 'sparse_conciergeservice', 'sparse_condofinishes', 'sparse_cook', 'sparse_courtyard', 'sparse_deck', 'sparse_decorativefireplace', 'sparse_diningroom', 'sparse_dishwasher', 'sparse_dogsallowed', 'sparse_doorman', 'sparse_drycleaningservice', 'sparse_dryer', 'sparse_dryerinbuilding', 'sparse_dryerinunit', 'sparse_duplex', 'sparse_eatinkitchen', 'sparse_elev', 'sparse_elevator', 'sparse_elevbldg', 'sparse_exclusive', 'sparse_exposedbrick', 'sparse_fireplace', 'sparse_fireplaces', 'sparse_fitness', 'sparse_fitnesscenter', 'sparse_flex2', 'sparse_flex3', 'sparse_ftdoorman', 'sparse_fullservicegarage', 'sparse_fulltimedoorman', 'sparse_furnished', 'sparse_garage', 'sparse_garden', 'sparse_gourmetkitchen', 'sparse_granitecountertops', 'sparse_granitekitchen', 'sparse_greenbuilding', 'sparse_guarantorsaccepted', 'sparse_gutrenovated', 'sparse_gym', 'sparse_gyminbuilding', 'sparse_hardwood', 'sparse_hardwoodfloors', 'sparse_healthclub', 'sparse_highceiling', 'sparse_highceilings', 'sparse_highrise', 'sparse_highspeedinternet', 'sparse_hirise', 'sparse_housekeeping', 'sparse_indoorpool', 'sparse_inunitwasher', 'sparse_largelivingroom', 'sparse_laundry', 'sparse_laundryinbuilding', 'sparse_laundryinunit', 'sparse_laundryonfloor', 'sparse_laundryroom', 'sparse_light', 'sparse_live', 'sparse_liveinsuper', 'sparse_liveinsuperintendent', 'sparse_lndrybldg', 'sparse_loft', 'sparse_lounge', 'sparse_loungeroom', 'sparse_lowrise', 'sparse_luxurybuilding', 'sparse_mailroom', 'sparse_marblebath', 'sparse_marblebathroom', 'sparse_microwave', 'sparse_midrise', 'sparse_mrcleanapproved', 'sparse_multilevel', 'sparse_new', 'sparse_newconstruction', 'sparse_newlyrenovated', 'sparse_nofee', 'sparse_nopets', 'sparse_nursery', 'sparse_onsitegarage', 'sparse_onsitelaundry', 'sparse_onsiteparking', 'sparse_onsiteparkingavailable', 'sparse_onsiteparkinglot', 'sparse_onsitesuper', 'sparse_ornateprewardetails', 'sparse_outdoorareas', 'sparse_outdoorentertainmentspace', 'sparse_outdoorpool', 'sparse_outdoorspace', 'sparse_packageroom', 'sparse_parking', 'sparse_parkingspace', 'sparse_patio', 'sparse_penthouse', 'sparse_petfriendly', 'sparse_pets', 'sparse_petsallowed', 'sparse_petsok', 'sparse_petsonapproval', 'sparse_photos', 'sparse_playroom', 'sparse_pool', 'sparse_postwar', 'sparse_prewar', 'sparse_privatebackyard', 'sparse_privatebalcony', 'sparse_privatedeck', 'sparse_privatelaundryroomoneveryfloor', 'sparse_privateoutdoorspace', 'sparse_privateparking', 'sparse_privateroofdeck', 'sparse_privateterrace', 'sparse_publicoutdoor', 'sparse_reducedfee', 'sparse_renovated', 'sparse_residentsgarden', 'sparse_residentslounge', 'sparse_roofaccess', 'sparse_roofdeck', 'sparse_rooftopdeck', 'sparse_rooftopterrace', 'sparse_roomyclosets', 'sparse_satellitetv', 'sparse_sauna', 'sparse_screeningroom', 'sparse_sharedbackyard', 'sparse_sharesok', 'sparse_shorttermallowed', 'sparse_simplex', 'sparse_skitchen', 'sparse_skylight', 'sparse_splayroom', 'sparse_sskitchen', 'sparse_stainlesssteelappliances', 'sparse_stepstothepark', 'sparse_storage', 'sparse_storagefacilitiesavailable', 'sparse_storageroom', 'sparse_sublet', 'sparse_subway', 'sparse_sundeck', 'sparse_swimmingpool', 'sparse_tenantlounge', 'sparse_terrace', 'sparse_terraces', 'sparse_tonsofnaturallight', 'sparse_valet', 'sparse_valetparking', 'sparse_valetservices', 'sparse_valetservicesincludingdrycleaning', 'sparse_videointercom', 'sparse_view', 'sparse_virtualdoorman', 'sparse_walkincloset', 'sparse_wallsofwindows', 'sparse_washer', 'sparse_washerinunit', 'sparse_wheelchairaccess', 'sparse_wheelchairramp', 'sparse_wifi', 'sparse_wifiaccess', 'sparse_work', 'street', 'west', 'worth_by_BoroCode', 'worth_by_NTACode', 'worth_by_address1']
sparse_cols = [x[0] for x in column_usefulness if x[0].startswith('sparse')][:15]
non_sparse_cols = [x[0] for x in column_usefulness if not x[0].startswith('sparse')]

# Add a bunch of features to train_X

In [106]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.neighbors import DistanceMetric

In [140]:
train_X = pd.read_json('data/train_with_leaky_cv.json')
train_y = pd.read_json('data/train_interest.json')[0]
columns = train_X.columns
train_X_scaled = pd.DataFrame(StandardScaler().fit_transform(train_X), columns=columns)
train_X_with_layer_1 = pd.read_json('data/train_scaled_with_leaky_and_added_features_cv.json')

In [141]:
knn_features = ['bc_price', 'manager_worths', 'latitude', 'longitude', 'desc_words_length', 'Yday', 'building_worths', 'price_per_bedroom']
knn_n_closest = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
n_components = 3
ridge_columns = ['layer1_ridge_predictions']
pca_columns = ['layer1_pca_{}'.format(i) for i in range(n_components)]
knn_columns = ['layer1_knn_{}_{}'.format(n_neighbors, category) for category in ['high', 'medium', 'low'] for n_neighbors in knn_n_closest]
rf_columns = ['layer1_rf_{}_{}_predictions'.format(criterion, category) for category in ['high', 'medium', 'low'] for criterion in ['gini', 'entropy']]
k_means_columns = ['layer1_k_means_{}'.format(n_clusters) for n_clusters in [2, 4, 8]]
new_columns_generated = ridge_columns + pca_columns + knn_columns + rf_columns + k_means_columns
for column in knn_columns:
    train_X_with_layer_1[column] = 0

In [142]:
#%%time
#PCA

pca_cols = ['layer1_pca_{}'.format(i) for i in range(n_components)]
pca = PCA(n_components=n_components)
pca_representation = pca.fit_transform(train_X_scaled)
train_X_with_layer_1.loc[:, pca_cols] = pca_representation


#tsne = TSNE(n_components = 2)
#tsne_representation = tsne.fit_transform(train_X_scaled[knn_features[:15]][:5000])

folds = 5
full_length = len(train_X_scaled)
entries = int(full_length / folds)
full_index = train_X_scaled.index
fold_indices = []
fold_indices_keep = []
fold_indices_throw = []
for column in leaky:
    train_X_scaled[column] = 0
for i in range(folds):
    print ('Fold {}'.format(i))
    indices = sorted(random.sample(list(full_index), entries))
    fold_indices.append(indices)
    full_index = full_index.drop(indices)

    full_index_keep = np.zeros((full_length), dtype=np.bool)
    full_index_throw = np.ones((full_length), dtype=np.bool)
    for j in fold_indices[i]:
        full_index_keep[j] = True
        full_index_throw[j] = False
    fold_indices_throw.append(full_index_throw)
    fold_indices_keep.append(full_index_keep)
    
    xtr = train_X_scaled[full_index_throw]
    ytr = train_y[full_index_throw]
    xte = train_X_scaled[full_index_keep]
    
    for n_neighbors in knn_n_closest:
        #knn_dist = DistanceMetric.get_metric('mahalanobis', V=np.eye(len(knn_n_closest)))
        knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='manhattan') # metric='manhattan'
        knn.fit(train_X_scaled[knn_features][full_index_throw], train_y[full_index_throw])
        knn_pred = knn.predict_proba(train_X_scaled[knn_features][full_index_keep])
        new_cols = ['layer1_knn_{}_{}'.format(n_neighbors, category) for category in ['high', 'medium', 'low']]
        train_X_with_layer_1.loc[indices, new_cols] = knn_pred
    
    '''
    ridge = Ridge()
    ridge.fit(xtr, ytr)
    ridge_predictions = ridge.predict(xte)
    train_X_with_layer_1.loc[indices, 'layer1_ridge_predictions'] = ridge_predictions
    
    for criterion in ['gini', 'entropy']:
        rf = RandomForestClassifier(
            n_estimators=25,
            criterion=criterion,
            max_features='auto',
            min_samples_split=20)

        rf.fit(xtr, ytr)
        rf_predictions = rf.predict_proba(xte)
        new_cols = ['layer1_rf_{}_{}_predictions'.format(criterion, category) for category in ['high', 'medium', 'low']]
        train_X_with_layer_1.loc[indices, new_cols] = rf_predictions
    
    for n_clusters in [2, 4, 8]:
        k_means = KMeans(n_clusters=n_clusters)
        k_means.fit(xtr[knn_features], ytr)
        k_means_indices = k_means.predict(xte[knn_features])
        train_X_with_layer_1.loc[indices, 'layer1_k_means_{}'.format(n_clusters)] = k_means_indices
    
    for n_neighbors in [2, 4, 8, 16, 64, 256, 1024]:
        knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='manhattan') # mahalanobis
        knn.fit(train_X_scaled[knn_features][full_index_throw], train_y[full_index_throw])
        knn_pred = knn.predict(train_X_scaled[knn_features][full_index_keep])
        train_X_with_layer_1.loc[indices, 'layer1_knn_{}'.format(n_neighbors)] = knn_pred
    '''

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [50]:
train_X_with_layer_1.to_json('data/train_scaled_with_leaky_and_added_features_cv.json', orient='records')

In [132]:
for i in range(5):
    test_nn_with_columns(train_X_with_layer_1, train_y, all_columns + [x for x in knn_columns if int(x.split('_')[2]) > 128])

epoch 50: loss = 0.5914606363581266, val = 0.5958407469459747
epoch 100: loss = 0.568360930373012, val = 0.5813010576421002
epoch 150: loss = 0.5530395628425995, val = 0.5782067945390374
epoch 200: loss = 0.5436676874319609, val = 0.5766880485184055
epoch 250: loss = 0.534290305472608, val = 0.5763892252067632
0.57560832226
epoch 50: loss = 0.5907035735841637, val = 0.5942039608588229
epoch 100: loss = 0.564317243756355, val = 0.5821416626913386
epoch 150: loss = 0.5488034374498629, val = 0.5774871603772829
epoch 200: loss = 0.539257666988917, val = 0.57818092279949
0.576082880874
epoch 50: loss = 0.5855864123006742, val = 0.5909427438317174
epoch 100: loss = 0.5643452853375035, val = 0.5818857556997927
epoch 150: loss = 0.5508964615383598, val = 0.579674539863242
0.578340048417
epoch 50: loss = 0.5909714835250617, val = 0.5927352374928333
epoch 100: loss = 0.5665478209976555, val = 0.5811703168876284
epoch 150: loss = 0.5495527357679457, val = 0.5770774268025729
epoch 200: loss = 0.54

In [ ]:
[x for x in knn_columns if int(x.split('_')[2]) > 128]

In [146]:
# after adding others
for i in range(5):
    test_nn_with_columns(train_X_with_layer_1, train_y, all_columns + pca_cols)
# 0.5761ish

epoch 50: loss = 0.5938834688481451, val = 0.5979886952202486
epoch 100: loss = 0.5703038286433625, val = 0.5848430726145167
epoch 150: loss = 0.5516893069625719, val = 0.5780559380333898
epoch 200: loss = 0.5443068869645845, val = 0.5776463041991154
epoch 250: loss = 0.5371102081381174, val = 0.5770619802042892
epoch 300: loss = 0.529178605043193, val = 0.5779906169273301
0.575908581058
epoch 50: loss = 0.5957561787088023, val = 0.6010609661514436
epoch 100: loss = 0.5700007426318121, val = 0.5846389501557433
epoch 150: loss = 0.551885164965363, val = 0.579517080092434
epoch 200: loss = 0.5435316820433739, val = 0.5773379306755339
0.576451235521
epoch 50: loss = 0.5939590478340205, val = 0.5995870211519014
epoch 100: loss = 0.5693623903324985, val = 0.5835657462170917
epoch 150: loss = 0.5535141397586626, val = 0.5789996878720891
epoch 200: loss = 0.5406997339255408, val = 0.5785544408664341
epoch 250: loss = 0.5346248421083105, val = 0.5785859360902038
0.576419783162
epoch 50: loss =

In [ ]:
%%time

SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

extra_cols_to_use = [x for x in knn_columns if int(x.split('_')[2]) > 128] + rf_columns + pca_columns
dtrain = xgb.DMatrix(data=train_X_with_layer_1[all_columns + extra_cols_to_use], label=train_y)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

[0]	train-mlogloss:1.09099+2.86744e-05	test-mlogloss:1.09118+3.23144e-05
[200]	train-mlogloss:0.574909+0.00234793	test-mlogloss:0.608062+0.00333607
[400]	train-mlogloss:0.501247+0.00298666	test-mlogloss:0.559249+0.00449265
[600]	train-mlogloss:0.469505+0.00322885	test-mlogloss:0.549452+0.00487377
[800]	train-mlogloss:0.445462+0.00329606	test-mlogloss:0.545648+0.00518052
[1000]	train-mlogloss:0.424577+0.00313616	test-mlogloss:0.543703+0.00533344
[1200]	train-mlogloss:0.405652+0.00262246	test-mlogloss:0.542455+0.00544267
[1400]	train-mlogloss:0.388198+0.00251352	test-mlogloss:0.541651+0.00548474
[1600]	train-mlogloss:0.372008+0.00211506	test-mlogloss:0.541326+0.00552556
[1800]	train-mlogloss:0.356742+0.00195141	test-mlogloss:0.541163+0.00552586
0.541107666667
1867
CPU times: user 2h 9min 27s, sys: 3min 24s, total: 2h 12min 52s
Wall time: 40min 13s


In [ ]:
%%time

SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

extra_cols_to_use = knn_columns + rf_columns + pca_columns
dtrain = xgb.DMatrix(data=train_X_with_layer_1[all_columns + extra_cols_to_use], label=train_y)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

[0]	train-mlogloss:1.09099+3.72499e-05	test-mlogloss:1.0912+1.666e-05
[200]	train-mlogloss:0.574327+0.00236692	test-mlogloss:0.60816+0.0033186
[400]	train-mlogloss:0.499712+0.00309028	test-mlogloss:0.559459+0.00472332
[600]	train-mlogloss:0.46717+0.00367726	test-mlogloss:0.549559+0.00507385
[800]	train-mlogloss:0.442664+0.00379257	test-mlogloss:0.545771+0.00517363
[1000]	train-mlogloss:0.421248+0.00340619	test-mlogloss:0.543869+0.00533699


In [ ]:
%%time

SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

extra_cols_to_use = knn_columns + rf_columns + pca_columns + k_means_columns
dtrain = xgb.DMatrix(data=train_X_with_layer_1[all_columns + extra_cols_to_use], label=train_y)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

In [ ]:
%%time

SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

extra_cols_to_use = knn_columns + rf_columns + pca_columns + ridge_columns
dtrain = xgb.DMatrix(data=train_X_with_layer_1[all_columns + extra_cols_to_use], label=train_y)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

In [ ]:
%%time

SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

extra_cols_to_use = [x for x in train_X_with_layer_1.columns if x.startswith('layer1_knn')] + rf_columns + pca_columns + ridge_columns + k_means_columns
dtrain = xgb.DMatrix(data=train_X_with_layer_1[all_columns + extra_cols_to_use], label=train_y)

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

In [59]:
# after adding others
for i in range(5):
    test_nn_with_columns(train_X, train_y, all_columns)
# 0.5761ish

epoch 50: loss = 0.5891488493100785, val = 0.5963193890939831
epoch 100: loss = 0.5633413129618845, val = 0.5837058686305724
epoch 150: loss = 0.5528043889919517, val = 0.5779784439371053
epoch 200: loss = 0.5419338218633879, val = 0.5762588906121962
0.575842619402
epoch 50: loss = 0.5988605105555501, val = 0.6008190141541198
epoch 100: loss = 0.5726395071071351, val = 0.5853478953466943
epoch 150: loss = 0.5545519629186689, val = 0.5786513815457169
epoch 200: loss = 0.5445255741711316, val = 0.577489306096185
0.576257448063
epoch 50: loss = 0.5970482967136526, val = 0.6019688100613078
epoch 100: loss = 0.5716081300411502, val = 0.585956075933923
epoch 150: loss = 0.5586140227548408, val = 0.5794861443735712
epoch 200: loss = 0.544868941424815, val = 0.5781161536233587
epoch 250: loss = 0.5349255880409272, val = 0.5778112565414081
0.576570762645
epoch 50: loss = 0.5934883482279587, val = 0.5983995749452238
epoch 100: loss = 0.5701759461581807, val = 0.5851851115738443
epoch 150: loss =

In [67]:
# after adding others using all columns
for i in range(5):
    test_nn_with_columns(train_X, train_y, all_columns + new_columns_generated)
# 0.5698982835840147

epoch 50: loss = 0.5660618697845898, val = 0.5738794634921525
epoch 100: loss = 0.5476543278769387, val = 0.5692151620836733
0.568399167234
epoch 50: loss = 0.5654138053050335, val = 0.5748133755692556
epoch 100: loss = 0.5478983607211402, val = 0.570332626475954
0.570071592402
epoch 50: loss = 0.5674681716364117, val = 0.5733706152097084
epoch 100: loss = 0.5466272889740071, val = 0.5702550949975377
0.569464498692
epoch 50: loss = 0.5714001220983553, val = 0.5742717927134893
epoch 100: loss = 0.5496349390401906, val = 0.5703977996670344
0.569792896392
epoch 50: loss = 0.5662333334826274, val = 0.5737657303515623
epoch 100: loss = 0.5473061113198573, val = 0.5703907628309826
epoch 150: loss = 0.5327434986531847, val = 0.5713913049491497
0.569260921208


In [60]:
for i in range(5):
    test_nn_with_columns(train_X, train_y, all_columns + [x for x in new_columns_generated if x.startswith('layer1_pca')])
# 0.5759712475000001

epoch 50: loss = 0.5968368610190747, val = 0.5983693936903408
epoch 100: loss = 0.5712352800570232, val = 0.5848432007611213
epoch 150: loss = 0.5580001741948851, val = 0.5787365255295545
epoch 200: loss = 0.5492674991075408, val = 0.5781092188369514
epoch 250: loss = 0.5349007071935745, val = 0.5770544120665984
0.576137658997
epoch 50: loss = 0.5948694923302533, val = 0.6006885247169467
epoch 100: loss = 0.5696178057652763, val = 0.5842879144506568
epoch 150: loss = 0.554824017569203, val = 0.5798235199481978
epoch 200: loss = 0.546429100270827, val = 0.577611118289365
0.576528663307
epoch 50: loss = 0.5957544322833704, val = 0.6000883799450268
epoch 100: loss = 0.574421041257675, val = 0.5853944098974554
epoch 150: loss = 0.554906663914879, val = 0.5792153256796306
epoch 200: loss = 0.5423362196271189, val = 0.5782430684630266
epoch 250: loss = 0.5353194022736467, val = 0.5760248840811728
0.575659649132
epoch 50: loss = 0.59418502251436, val = 0.5983718875007549
epoch 100: loss = 0.5

In [61]:
for i in range(5):
    test_nn_with_columns(train_X, train_y, all_columns + [x for x in new_columns_generated if x.startswith('layer1_knn')])
# 0.577724697

epoch 50: loss = 0.5907334422552457, val = 0.5967496885223779
epoch 100: loss = 0.5670951068491515, val = 0.5832223275778726
epoch 150: loss = 0.5500290452880038, val = 0.5784954736406286
epoch 200: loss = 0.5379367678956557, val = 0.5774372410476643
0.576675237958
epoch 50: loss = 0.5864287301195095, val = 0.5981343093770968
epoch 100: loss = 0.5646792716825905, val = 0.5833248486007162
epoch 150: loss = 0.5471640735463668, val = 0.5787726602343304
epoch 200: loss = 0.5385090508343509, val = 0.5780075173417412
epoch 250: loss = 0.5280735885932056, val = 0.5783458846298295
0.576167545162
epoch 50: loss = 0.5934461387834576, val = 0.5984918959215368
epoch 100: loss = 0.5671759191289574, val = 0.5851840766907357
epoch 150: loss = 0.547754690401325, val = 0.5799696629525468
epoch 200: loss = 0.5397729572526071, val = 0.5780265740259171
epoch 250: loss = 0.5292247672447375, val = 0.5786475343842583
0.57731011351
epoch 50: loss = 0.5899822293452791, val = 0.5985474366870416
epoch 100: loss 

In [62]:
for i in range(5):
    test_nn_with_columns(train_X, train_y, all_columns + [x for x in new_columns_generated if x.startswith('layer1_rf')])
# 0.56814564182608

epoch 50: loss = 0.5682796702918547, val = 0.5714792323027462
epoch 100: loss = 0.5511596248003912, val = 0.5693363437758483
epoch 150: loss = 0.5404633747485917, val = 0.569118391474249
0.567836367203
epoch 50: loss = 0.5657308571257622, val = 0.5723934772818879
epoch 100: loss = 0.549216198268699, val = 0.5686539264103256
epoch 150: loss = 0.5365811970636242, val = 0.5692361380779911
0.567620550835
epoch 50: loss = 0.5682518029094431, val = 0.5728822978626177
epoch 100: loss = 0.5504466508836087, val = 0.5691213133385742
0.568624672508
epoch 50: loss = 0.5668625585835892, val = 0.5729637192759991
epoch 100: loss = 0.5491214824913193, val = 0.569128129128255
epoch 150: loss = 0.5370663559829074, val = 0.5686327860563689
0.567867505089
epoch 50: loss = 0.567294668367399, val = 0.5739266412399986
epoch 100: loss = 0.5500730488976918, val = 0.5695023835236761
epoch 150: loss = 0.5388992663967865, val = 0.5699179627594289
0.56807929753


In [63]:
for i in range(5):
    test_nn_with_columns(train_X, train_y, all_columns + [x for x in new_columns_generated if x.startswith('layer1_k_means')])
# 0.5784255874603335

epoch 50: loss = 0.5929667140165115, val = 0.5996124723738684
epoch 100: loss = 0.5663412545858033, val = 0.5845144609999939
epoch 150: loss = 0.5503417076213359, val = 0.5779046327295982
0.577296035055
epoch 50: loss = 0.5929926436956255, val = 0.5991310493005774
epoch 100: loss = 0.5687427557113137, val = 0.5856202454906284
epoch 150: loss = 0.54791703039381, val = 0.5795868810735465
epoch 200: loss = 0.5395159363205573, val = 0.5790985688859436
0.577202777553
epoch 50: loss = 0.5951749289796515, val = 0.6005283730848234
epoch 100: loss = 0.5687148210929177, val = 0.5866093116615093
epoch 150: loss = 0.5505405192730872, val = 0.5808669920194901
epoch 200: loss = 0.5422069537309179, val = 0.5800369410697249
0.578376408242
epoch 50: loss = 0.593097744737741, val = 0.6007689756345355
epoch 100: loss = 0.567927184957491, val = 0.5856907262004433
epoch 150: loss = 0.5510727192959162, val = 0.5802431579520432
epoch 200: loss = 0.540886766038271, val = 0.5797369761199537
0.578568421474
epoc

In [64]:
for i in range(5):
    test_nn_with_columns(train_X, train_y, all_columns + [x for x in new_columns_generated if x.startswith('layer1_ridge')])
# 0.5784149550699805

epoch 50: loss = 0.593501963488266, val = 0.6007780395212697
epoch 100: loss = 0.568974771543177, val = 0.5851150762515007
epoch 150: loss = 0.5527953938210926, val = 0.5792887583797688
epoch 200: loss = 0.5402982440046085, val = 0.5776843522912506
0.576746761903
epoch 50: loss = 0.5894670833540084, val = 0.5975216648763767
epoch 100: loss = 0.5641740372123354, val = 0.5833631754984324
epoch 150: loss = 0.5490681999002947, val = 0.5792636525826123
epoch 200: loss = 0.5427933462053101, val = 0.5776859821579912
epoch 250: loss = 0.5346804329098969, val = 0.5783245299445653
0.577357405282
epoch 50: loss = 0.5930712994077562, val = 0.6002991233213656
epoch 100: loss = 0.5705091002089616, val = 0.5842558635577794
epoch 150: loss = 0.5533849896675198, val = 0.5807215348308796
epoch 200: loss = 0.5392204656743691, val = 0.5780524985777499
epoch 250: loss = 0.5313960316019248, val = 0.5781242910438621
0.577002162568
epoch 50: loss = 0.592848019352053, val = 0.5984043470463233
epoch 100: loss =

In [65]:
import xgboost as xgb
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])

In [66]:
SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

dtrain = xgb.DMatrix(data=train_X, label=train_y)
bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

[0]	train-mlogloss:1.09102+3.71573e-05	test-mlogloss:1.09122+1.91369e-05
[200]	train-mlogloss:0.573924+0.00207436	test-mlogloss:0.607135+0.00328306
[400]	train-mlogloss:0.499542+0.00258495	test-mlogloss:0.55791+0.00442858
[600]	train-mlogloss:0.466972+0.00312354	test-mlogloss:0.547912+0.00461182
[800]	train-mlogloss:0.442694+0.0031566	test-mlogloss:0.544045+0.00467582
[1000]	train-mlogloss:0.422039+0.00339214	test-mlogloss:0.542069+0.00467361
[1200]	train-mlogloss:0.402974+0.00281161	test-mlogloss:0.540841+0.00466194
[1400]	train-mlogloss:0.385328+0.0026261	test-mlogloss:0.540158+0.00472587
[1600]	train-mlogloss:0.368956+0.00246329	test-mlogloss:0.539705+0.00470666
[1800]	train-mlogloss:0.353754+0.00244792	test-mlogloss:0.539548+0.00468775
0.539502
1863


# Train Models

In [6]:
def create_model(xtr):
    model = Sequential()
    
    model.add(Dense(500, input_dim=xtr.shape[1], init = 'he_normal', activation='sigmoid'))
    #model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.75))
    model.add(PReLU())

    model.add(Dense(100, init = 'he_normal', activation='sigmoid'))
    model.add(BatchNormalization())    
    model.add(Dropout(0.75))
    model.add(PReLU())

    model.add(Dense(3, init = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

In [58]:
def test_nn_with_columns(X, y, columns_to_include):
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    ntrain = int(len(train_X) * 0.75)

    xtr = X_scaled[columns_to_include][:ntrain]
    ytr = y[:ntrain].values
    ytr_one_hot = np.eye(3)[ytr]
    xte = X_scaled[columns_to_include][ntrain:]
    yte = y[ntrain:].values
    yte_one_hot = np.eye(3)[yte]

    filepath = 'data/nn_weights'

    logger = NBatchLogger()
    early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
    model = create_model(xtr)
    model.fit(xtr.values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte.values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])
    model = create_model(xtr)
    model.load_weights(filepath)
    print (log_loss(yte_one_hot, model.predict(xte.values)))

In [45]:
#new_vects = ['tf_idf_' + x[7:] for x in sparse_cols]
#sparse = processed_train[sparse_cols]
#tf_idfs = TfidfTransformer().fit_transform(sparse)
#tf_idfs_df = pd.DataFrame(tf_idfs.toarray(), columns=new_vects)
#processed_train = processed_train.join(tf_idfs_df, how='left')

# Ridge Lasso Elastic

In [308]:
las = Lasso()
enet = ElasticNet()
las.fit(xte, yte)
enet.fit(xte, yte)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [309]:
print (las.score(xte, yte))
predictions = las.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [310]:
print (enet.score(xte, yte))
predictions = enet.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [42]:
for coef, column in zip(enet.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

In [43]:
for coef, column in zip(las.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

# TREES

In [322]:
test = ExtraTreesClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, predictions)))

0.5843451108603969 (test loss 0.3912286072900344)


In [323]:
test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_ent_predictions), log_loss(ytr_one_hot, predictions)))

0.5841850290743527 (test loss 0.3945679429419886)


In [335]:
test = RandomForestClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
rf_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_predictions), log_loss(ytr_one_hot, predictions)))

0.5693372215730407 (test loss 0.3780482702094549)


In [336]:
test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
rf_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions), log_loss(ytr_one_hot, predictions)))

0.5668175898671493 (test loss 0.3773447326368733)


In [337]:
pred_mixed = (0.01 * et_predictions + 0.01 * et_ent_predictions + 0.49 * rf_predictions + 0.49 * rf_ent_predictions)
log_loss(yte_one_hot, pred_mixed)

0.56688904952650898

In [12]:
%%time
test = KNeighborsClassifier(n_neighbors=2)
test.fit(xtr[non_sparse_cols], ytr)

predictions = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions = test.predict_proba(xte[non_sparse_cols])

7.076257047822155 (test loss 0.23129250627049086)
CPU times: user 3min 15s, sys: 1.48 s, total: 3min 16s
Wall time: 3min 20s


In [13]:
%%time
test = KNeighborsClassifier(n_neighbors=5)
test.fit(xtr[non_sparse_cols], ytr)

predictions_2 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_2 = test.predict_proba(xte[non_sparse_cols])

2.997161309463761 (test loss 0.4189513500335195)
CPU times: user 3min 36s, sys: 1.02 s, total: 3min 37s
Wall time: 3min 38s


In [14]:
%%time
test = KNeighborsClassifier(n_neighbors=10)
test.fit(xtr[non_sparse_cols], ytr)

predictions_3 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_3 = test.predict_proba(xte[non_sparse_cols])

1.5474359459916005 (test loss 0.4994428601539661)
CPU times: user 3min 48s, sys: 839 ms, total: 3min 49s
Wall time: 3min 50s


In [21]:
%%time
test = KNeighborsClassifier(n_neighbors=200)
test.fit(xtr[non_sparse_cols], ytr)

predictions_6 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_6 = test.predict_proba(xte[non_sparse_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions_6), log_loss(ytr_one_hot, predictions_6)))

0.6344317626130038 (test loss 0.6195293255389913)
CPU times: user 4min 34s, sys: 1.19 s, total: 4min 35s
Wall time: 4min 36s


In [ ]:
for feat, importance in zip(processed_train[cols].columns, clf.feature_importances_):
    if not np.isnan(importance) and importance > 0:
        print (feat, ": ", importance)

In [345]:
print ("nn: ", log_loss(yte_one_hot, nn_predictions))
print ("ab: ", log_loss(yte_one_hot, pred_mixed))
frac_nn = 0.2
averaged_preds = (nn_predictions * frac_nn) + (pred_mixed * (1 - frac_nn))
print ("combo: ", log_loss(yte_one_hot, averaged_preds))

nn:  0.563593691555
ab:  0.566889049527
combo:  0.55827286106


In [ ]:
train_X = pd.read_json('data/train_leaky.json')
train_y = pd.read_json('data/train_interest.json')[0]
scaler = StandardScaler().fit(train_X)
train_X = pd.DataFrame(scaler.transform(train_X), columns=columns)
test_X = pd.read_json('data/test_leaky.json')
test_X = pd.DataFrame(scaler.transform(test_X), columns=columns)
listing_id = pd.read_json('data/test_ids.json').values
preds = pd.read_csv('data/my_best_preds.csv')

In [295]:
preds = pd.read_csv('data/my_best_preds.csv')

In [313]:
et_clf = ExtraTreesClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='auto',
    min_samples_split=20)
et_clf.fit(train_X, train_y)          
et_predictions = et_clf.predict_proba(test_X)

In [314]:
et_clf_ent = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)
et_clf_ent.fit(train_X, train_y)          
et_ent_predictions = et_clf_ent.predict_proba(test_X)

In [316]:
rf_clf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_split=20)
rf_clf.fit(train_X, train_y)          
rf_predictions = rf_clf.predict_proba(test_X)

In [317]:
rf_clf_ent = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    min_samples_split=20)
rf_clf_ent.fit(train_X, train_y)          
rf_ent_predictions = rf_clf_ent.predict_proba(test_X)

In [259]:
nn_predictions = model.predict(test_X.values)

In [297]:
xgb_predictions = preds[['high', 'medium', 'low']].values

In [303]:
new_preds = (0.8 * xgb_predictions) + (0.15 * nn_predictions) + (0.05 * et_predictions)

In [304]:
new_preds_df = pd.DataFrame(new_preds, columns=['high', 'medium', 'low'])
new_preds_df['listing_id'] = preds['listing_id']

In [306]:
new_preds_df.to_csv('data/combined_preds.csv', index=None)